In [ ]:
import tensorflow as tf 
import os
from random import shuffle
import numpy as np

2022-03-21 08:29:47.687257: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [ ]:
os.chdir('/root/fish_class')
working_directory = os.getcwd()
print("working directory:", working_directory)

working directory: /root/fish_class


1. Loading Data and Preprocessing

In [ ]:
# Potentially remove this and try again .. 
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
   rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,     
    zoom_range=0.2,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(                                                    
    rescale=1./255 # Apply same normalization, not performing other preprocessing steps
)

In [ ]:
# BATCH SIZE WAS ORIGINALLY 32
train_images = train_generator.flow_from_directory(
    directory= './Data/Train_Val',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=24,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_directory(
    directory= './Data/Train_Val',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=24,
    shuffle=True,
    seed=42,
    subset='validation' # Will only take 20% of the total data as the validation data
)

test_images = test_generator.flow_from_directory(
    directory= './Data/Test',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=24,
    shuffle=False, # We need to be able to generate metrics at the end
    seed=42
)

Found 7218 images belonging to 9 classes.
Found 1803 images belonging to 9 classes.
Found 498 images belonging to 9 classes.


In [ ]:
print("Training image shape:", train_images.image_shape)
print("Validation image shape:", val_images.image_shape)
print("Test image shape:", test_images.image_shape)

Training image shape: (224, 224, 3)
Validation image shape: (224, 224, 3)
Test image shape: (224, 224, 3)


In [ ]:
train_images.class_indices

{'Black Sea Sprat': 0,
 'Gilt-Head Bream': 1,
 'Hourse Mackerel': 2,
 'Red Mullet': 3,
 'Red Sea Bream': 4,
 'Sea Bass': 5,
 'Shrimp': 6,
 'Striped Red Mullet': 7,
 'Trout': 8}

In [ ]:
val_images.class_indices

{'Black Sea Sprat': 0,
 'Gilt-Head Bream': 1,
 'Hourse Mackerel': 2,
 'Red Mullet': 3,
 'Red Sea Bream': 4,
 'Sea Bass': 5,
 'Shrimp': 6,
 'Striped Red Mullet': 7,
 'Trout': 8}

In [ ]:
test_images.class_indices

{'Black Sea Sprat': 0,
 'Gilt Head Bream': 1,
 'Horse Mackerel': 2,
 'Red Mullet': 3,
 'Red Sea Bream': 4,
 'Sea Bass': 5,
 'Shrimp': 6,
 'Striped Red Mullet': 7,
 'Trout': 8}

In [ ]:
import tensorflow.keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

2. Defining VGG16 (CNN) Architecture

In [ ]:
model = tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(32, (3,3), activation='relu',  input_shape=(224,224,3), kernel_regularizer=tf.keras.regularizers.l2(l2=0.001)),
    tf.keras.layers.MaxPool2D(pool_size = (2,2)),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.01)),
    tf.keras.layers.MaxPool2D(pool_size = (2,2)),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.001)),
    tf.keras.layers.MaxPool2D(pool_size = (2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.01)),
    tf.keras.layers.MaxPool2D(pool_size = (2,2)),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.35),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.15),
    tf.keras.layers.Dense(9, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam()

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

2022-03-21 08:29:50.497378: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-21 08:29:50.498924: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-03-21 08:29:50.528816: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-21 08:29:50.529654: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2022-03-21 08:29:50.529693: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-03-21 08:29:50.532643: I tensorflow/stream_executor/platform/default/dso_lo

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 64)        0

3. Defining Schedulers and Callbacks

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 10) # Fine tune
checkpoint_path = "ENEL645_FinalProject_FishClassification/training_2_rof/cp.ckpt"
monitor = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss',
                                             verbose=1,save_best_only=True,
                                             save_weights_only=True,
                                             mode='min') # Only saves the best model (so far) in terms of min validation loss

def scheduler(epoch, lr):
    if epoch%10 == 0 and epoch!= 0:
        lr = lr/1.3
    return lr

lr_schedule = tf.keras.callbacks.LearningRateScheduler(scheduler,verbose = 0)
lr_schedule_on_plateau = ReduceLROnPlateau(monitor='val_loss', mode='min', factor=0.1, patience=5, min_lr=0.00000001, verbose=1)
callbacks = [early_stop, monitor, lr_schedule_on_plateau,lr_schedule]

4. Training Model

In [13]:
try:
    history = model.fit(
        train_images, 
        validation_data=val_images, 
        epochs=50, # Fine tune
        callbacks=callbacks
    )
except KeyboardInterrupt:
    print("\nmodel training terminated\n")

2022-03-21 08:30:15.719874: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-03-21 08:30:15.721127: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2300015000 Hz


Epoch 1/35


2022-03-21 08:30:16.763351: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-03-21 08:30:17.346950: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-03-21 08:30:18.264281: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2022-03-21 08:30:18.311612: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


301/301 [==============================] - 218s 689ms/step - loss: 2.5026 - accuracy: 0.1289 - val_loss: 1.8593 - val_accuracy: 0.2607

Epoch 00001: val_loss improved from inf to 1.85933, saving model to ENEL645_FinalProject_FishClassification/training_2_rof/cp.ckpt
Epoch 2/35
228/301 [=====================>........] - ETA: 37s - loss: 1.7805 - accuracy: 0.2539
model training terminated



In [132]:
np.save('ENEL645_FinalProject_FishClassification/history.npy', history.history)

In [ ]:
model.save('ENEL645_FinalProject_FishClassification/Model_rof')

In [ ]:
print("\n************************ COMPLETED TRAINING ************************")

5. Loading Best Model and Testing

In [349]:
model.load_weights(checkpoint_path)


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6aaaed87f0> and <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f6aa4fd5070>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6aaac831c0> and <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f6aaaed82b0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6aaac83280> and <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6aaac831c0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x7f6aa819e8e0> and <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6aaac83280>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x7f6aa819e

ValueError: Shapes (3, 3, 64, 32) and (3, 3, 64, 64) are incompatible

In [195]:
history=np.load('ENEL645_FinalProject_FishClassification/history.npy', allow_pickle='TRUE').item()
print("Best training results:\n", history)

Best training results:
 {'loss': [2.4210667610168457, 1.6860369443893433, 1.4987592697143555, 1.3576242923736572, 1.266735315322876, 1.2101818323135376, 1.1518763303756714, 1.107641339302063, 1.0793124437332153, 1.0533733367919922, 1.0100849866867065, 0.9996258020401001, 0.981090247631073, 0.8774905800819397, 0.8706485033035278, 0.8610906600952148, 0.8633388876914978, 0.8503754138946533, 0.8431152701377869, 0.8437260389328003, 0.8312422633171082, 0.8312739133834839, 0.8261463046073914, 0.8278459310531616, 0.8317890167236328, 0.8244032263755798, 0.8289649486541748, 0.8184067010879517, 0.8163051605224609, 0.8310553431510925, 0.8110599517822266, 0.8168381452560425, 0.8261783123016357, 0.7960702180862427, 0.808247447013855, 0.8139305114746094, 0.8101491928100586, 0.8136248588562012, 0.8225804567337036, 0.8263460397720337, 0.8186246752738953], 'accuracy': [0.20549464225769043, 0.36220279335975647, 0.43291857838630676, 0.4825674593448639, 0.5046199560165405, 0.5274115800857544, 0.55808794498

In [197]:
results = model.evaluate(test_images, verbose=1)

print("Categorical Cross Entropy: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

16/16 [==============================] - 28s 2s/step - loss: 1.4524 - accuracy: 0.6165
Categorical Cross Entropy: 1.45236
Test Accuracy: 61.65%


In [ ]:
print("\n************************ COMPLETED TESTING ************************")